In [ ]:
#!/usr/bin/env python
"""
STAGE 01 · DATA LOAD & PRE-PROCESSING
────────────────────────────────────
Reads the raw CSV, cleans it, and saves
   outputs_rff/event=<YEAR>/<RUN_TAG>/stage01/stage01_cleaned.csv

The orchestrator (or your own shell) must supply four ENV variables:
  • INPUT_CSV    – full path to the raw data file
  • OUTPUT_ROOT  – where the pipeline writes its outputs
  • SWAN_YEAR    – the crisis year we are working on (2000 / 2008 / 2020 …)
  • RUN_TAG      – any folder tag, usually today’s date (YYYY-MM-DD)

Everything else (date column name, filters, etc.) is read from
pipeline_config.yaml via pipeline_utils.load_cfg().
"""

from __future__ import annotations
from pathlib import Path
from datetime import datetime
import io, os, re, sys, logging

import pandas as pd
import numpy as np

from pipeline_utils import load_cfg, resolve_run_dir

# ───────────────────────────────────────────────
# 0 · CONFIG  &  RUN FOLDER   (Stage-01 only)
# ───────────────────────────────────────────────
from pathlib import Path
from datetime import datetime
import os

from pipeline_utils import load_cfg

CFG      = load_cfg()
EVENTS   = {str(k): v for k, v in CFG.get("events", {}).items()}   # NEW
DEFAULTS = CFG.get("defaults", {})

# event & run-tag
SWAN_YEAR = str(os.getenv("SWAN_YEAR") or next(iter(EVENTS)))      # safe fallback
RUN_TAG   = os.getenv("RUN_TAG") or datetime.today().strftime("%Y-%m-%d")

# paths
OUTPUT_ROOT = Path(os.getenv("OUTPUT_ROOT",
                             DEFAULTS.get("OUTPUT_ROOT", "outputs_rff")))
RUN_DIR  = OUTPUT_ROOT / f"event={SWAN_YEAR}" / RUN_TAG            # ← we CREATE it
RUN_DIR.mkdir(parents=True, exist_ok=True)

INPUT_CSV  = Path(os.getenv("INPUT_CSV", DEFAULTS.get("INPUT_CSV", ""))).expanduser()
OUTPUT_DIR = RUN_DIR / "stage01"; OUTPUT_DIR.mkdir(exist_ok=True)

DATE_COL = DEFAULTS.get("DATE_COL", "ReportDate")
ID_COL   = DEFAULTS.get("ID_COL",   "Symbol")
FILTERS  = {
    "pct_non_na": DEFAULTS.get("PCT_NON_NA", 95),
    "pct_zero":   DEFAULTS.get("PCT_ZERO",   98),
    "min_unique": DEFAULTS.get("MIN_UNIQUE", 10),
}

# ───────────────────────────────────────────────
# 1 · LOGGER
# ───────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-7s | %(message)s",
    handlers=[
        logging.FileHandler(OUTPUT_DIR / "stage01.log", mode="w", encoding="utf-8"),
        logging.StreamHandler(sys.stdout),
    ],
)
log = logging.getLogger(__name__)
log.info("========== STAGE 01 ==========")
log.info("RUN_DIR   : %s", RUN_DIR)
log.info("INPUT_CSV : %s", INPUT_CSV)
log.info("SWAN_YEAR : %s  |  RUN_TAG: %s", SWAN_YEAR, RUN_TAG)
log.info("DATE / ID : %s / %s", DATE_COL, ID_COL)
log.info("FILTERS   : %s", FILTERS)

# ───────────────────────────────────────────────
# 2 · LOAD RAW DATA
# ───────────────────────────────────────────────
df = pd.read_csv(INPUT_CSV, low_memory=False)
log.info("Rows loaded: %s", f"{len(df):,}")

# ───────────────────────────────────────────────
# 3 · DATE & ID CLEAN-UP
# ───────────────────────────────────────────────
df[DATE_COL] = (
    pd.to_datetime(df[DATE_COL], errors="coerce", dayfirst=True)
      .fillna(pd.to_datetime(df[DATE_COL], errors="coerce", dayfirst=False))
)
df[ID_COL]   = df[ID_COL].astype(str).str.strip().str.upper()

bad_dates = df[df[DATE_COL].isna()]
if not bad_dates.empty:
    bad_dates.to_csv(OUTPUT_DIR / "bad_dates.csv", index=False)
    log.warning("Bad dates → %d rows written to bad_dates.csv", len(bad_dates))

# ───────────────────────────────────────────────
# 4 · COERCE NUMERIC TEXT → FLOATS
# ───────────────────────────────────────────────
_num_rx = re.compile(r"[$€£,%]")
def to_num(series: pd.Series) -> pd.Series:
    if series.dtype != "object":
        return series
    out = pd.to_numeric(series.str.replace(_num_rx, "", regex=True), errors="coerce")
    # keep conversion only if ≥50 % values became numbers
    return out if out.notna().mean() >= 0.50 else series

df = df.apply(to_num)

# ───────────────────────────────────────────────
# 5 · BASIC FILTERS
# ───────────────────────────────────────────────
before = len(df)
df = df.dropna(subset=[ID_COL, DATE_COL])
log.info("After ID/date filter: %d rows (%.1f %%)", len(df), len(df)/before*100)

num_cols = df.select_dtypes(include=[np.number]).columns
meta = pd.DataFrame({
    "pct_non_na": df[num_cols].notna().mean()*100,
    "pct_zero":   (df[num_cols]==0).mean()*100,
    "n_unique":   df[num_cols].nunique(dropna=True),
})
good = (
    (meta["pct_non_na"] >= FILTERS["pct_non_na"]) &
    (meta["pct_zero"]   <  FILTERS["pct_zero"])   &
    (meta["n_unique"]   >= FILTERS["min_unique"])
)
drop_cols = list(meta.index[~good])
if drop_cols:
    df = df.drop(columns=drop_cols)
    log.info("Dropped %d noisy numeric columns", len(drop_cols))

# ───────────────────────────────────────────────
# 6 · KEEP LAST REPORT PER FIRM-YEAR
# ───────────────────────────────────────────────
df["Year"] = df[DATE_COL].dt.year
df = (
    df.sort_values(DATE_COL)
      .groupby([ID_COL, "Year"], as_index=False)
      .last()
)

# ───────────────────────────────────────────────
# 7 · SAVE RESULT   (patched: add _<SWAN_YEAR>)
# ───────────────────────────────────────────────
out_csv = OUTPUT_DIR / f"stage01_cleaned_{SWAN_YEAR}.csv"
df.to_csv(out_csv, index=False)

buf = io.StringIO(); df.info(buf=buf)
log.info("Final DataFrame info:\n%s", buf.getvalue())
log.info("Saved cleaned CSV → %s", out_csv)
log.info("✅ STAGE 01 complete")

2025-06-13 13:35:59,885 | INFO    | ==========  STAGE 01: DATA LOAD & PRE-PROCESSING ==========
2025-06-13 13:35:59,887 | INFO    | RUN_DIR        : outputs_rff\daily\2025-06-13
2025-06-13 13:35:59,888 | INFO    | INPUT_CSV      : C:\Users\Jason Pohl\OneDrive - Bond University\PhD\rff\NEW_DATA.csv
2025-06-13 13:35:59,891 | INFO    | SWAN_YEAR      : 2000  |  RUN_DATE: 2025-06-13
2025-06-13 13:35:59,892 | INFO    | DATE_COL / ID_COL   = ReportDate / Symbol
2025-06-13 13:35:59,894 | INFO    | FILTERS             = {'pct_non_na': 95, 'pct_zero': 98, 'min_unique': 10}
2025-06-13 13:36:01,680 | INFO    | Rows loaded: 55,800


C:\Users\Jason Pohl\AppData\Local\Temp\ipykernel_8172\649719724.py:81: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S.%f format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  pd.to_datetime(df[DATE_COL], errors="coerce", dayfirst=True)


2025-06-13 13:36:02,275 | INFO    | After ID/date filter: 55800 rows (100.0%)
2025-06-13 13:36:02,705 | INFO    | Dropped 94 low-quality numeric columns
2025-06-13 13:36:05,925 | INFO    | Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34862 entries, 0 to 34861
Columns: 164 entries, Symbol to ReportDate
dtypes: datetime64[ns](1), float64(148), int32(1), int64(4), object(10)
memory usage: 43.5+ MB

2025-06-13 13:36:05,926 | INFO    | Saved cleaned CSV → outputs_rff\daily\2025-06-13\stage01\stage01_cleaned.csv
2025-06-13 13:36:05,941 | INFO    | ✅ STAGE 01 complete — `data_stage_1` ready
